# Advanced CSP pipelines

This notebook implements more advanced CSP pipelines and tests their performance on the data from the database provided by [Kaya et al.](https://doi.org/10.1038/sdata.2018.211).
The knowledge and utilities obtained from the previous paper notebook 2 and the experimental notebooks four to five are used throughout this notebook.
Due to the small variations in accuracy found between the tested classifiers in the paper notebook 2, we opt to use an LDA approach in this noteboook. 
This has the pro that LDA doesn't actually require hyperparameter tuning, saving a lot of computational time.

This notebook works in an offline fashion and uses epochs with a length of 3 seconds.
This epoch starts 1 second before the visual queue was given, includes the 1 second the visual queue was shown and ends 1 second after the visual queue was hidden, totalling 3 seconds.
Baseline correction was done on the first second of the epoch, meaning the second before the visual queue was shown.
The effective training and testing are done in a half-second window, starting 0.1 seconds after the start of the visual queue.
A window of 0.5 seconds was chosen as it is a common size for sliding window approaches in online systems.
Alternative experiments include longer windows, such as 1.5 seconds windows.

Instructions on where to get the data are available on [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis). These instructions are under `bci-master-thesis/code/data/CLA/README.md`. We will use the utility file `bci-master-thesis/code/utils/CLA_dataset.py` to work with this data. The data was stored as FIF files, which are included in [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis).

<hr><hr>

## Table of Contents

- Checking requirements
   - Correct Anaconda environment
   - Correct module access
   - Correct file access
- Learned filter thresholds
   - Creation of custom SKLearn pipeline component for filter
   - Same subject, same session: LDA classifier
   - Same subject, new session: LDA classifier
   - New subject: LDA classifier
- FBCSP approaches
   - Same subject, same session: LDA classifier
   - Same subject, new session: LDA classifier
   - New subject: LDA classifier

<hr><hr>

## Checking requirements

### Correct Anaconda environment

The `bci-master-thesis` Anaconda environment should be active to ensure proper support. Installation instructions are available on [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis).

In [ ]:
####################################################
# CHECKING FOR RIGHT ANACONDA ENVIRONMENT
####################################################

import os
from platform import python_version
from pathlib import Path
from copy import copy

print(f"Active environment: {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Correct environment: {os.environ['CONDA_DEFAULT_ENV'] == 'bci-master-thesis'}")
print(f"\nPython version: {python_version()}")
print(f"Correct Python version: {python_version() == '3.8.10'}")

<hr>

### Correct module access

The following code block will load in all required modules.

In [ ]:
####################################################
# LOADING MODULES
####################################################

# Set logging level for MNE before loading MNE
os.environ['MNE_LOGGING_LEVEL'] = 'WARNING'

# Load util function file
import sys
sys.path.append('../utils')
import CLA_dataset

# IO functions
from IPython.utils import io
import copy

# Modules tailored for EEG data
import mne; print(f"MNE version (1.0.2 recommended): {mne.__version__}")
from mne.decoding import CSP

# ML libraries
import sklearn;  print(f"Scikit-learn version (1.0.2 recommended): {sklearn.__version__}")
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.base import BaseEstimator,TransformerMixin

# Custom SKlearn components
import custom_sklearn_components
from custom_sklearn_components import EpochsToFilteredData

# Data manipulation modules
import numpy as np; print(f"Numpy version (1.21.5 recommended): {np.__version__}")
import pandas as pd; print(f"Pandas version (1.4.1 recommended): {pd.__version__}")

# Plotting
import matplotlib; print(f"Matplotlib version (3.5.1 recommended): {matplotlib.__version__}")
import matplotlib.pyplot as plt

# Storing files
import pickle;  print(f"Pickle version (4.0 recommended): {pickle.format_version}")

<hr>

### Correct file access

As mentioned, this notebook uses a database provided by [Kaya et al](https://doi.org/10.1038/sdata.2018.211). The CLA dataset in particular. Instructions on where to get the data are available on [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis). These instructions are under `bci-master-thesis/code/data/CLA/README.md`. The following code block checks if all required files are available.

In [ ]:
####################################################
# CHECKING FILE ACCESS
####################################################

# Use util to determine if we have access
print("Full Matlab CLA file access: " + str(CLA_dataset.check_matlab_files_availability()))
print("Full MNE CLA file access: " + str(CLA_dataset.check_mne_files_availability()))

<hr><hr>

## Learned filter thresholds

The previous paper notebook, notebook `2-standard-csp-pipelines`, made use of a fixed filter aproach, namely an overlap-add FIR filter to obtain the frequencies 2Hz to 32Hz.
These thresholds will be learned in this experiment by using grid search for them, this should fit the frequencies to the subject(s) in question.
To do this, a custom sklearn pipeline component is made from the previously used filter.

<hr>

### Creation of custom SKLearn pipeline component for filter

Since we want to tune the lower and upper bound of the filter using gridsearch, we need to make the filter available as a function that is understood by SKLearn.
This creation is done in the provided util file `custom_sklearn_components`.
The below code block tests its implementation is correct.

In [ ]:
####################################################
# TEST FILTER
####################################################

with io.capture_output():
    # Get MNE raw object for latest recording of subject B
    mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id= "B")
    # Get epochs for that MNE raw
    mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                         start_offset= -1,
                                                         end_offset= 1,
                                                         baseline= (None, 0))
    # mne raw not needed anymore
    del mne_raw
    
    # Only keep epochs from the MI tasks
    mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
    
    # Load epochs into memory
    mne_epochs.load_data()

# Create copies for testing both approaches
manual_conversion = copy.deepcopy(mne_epochs)
automatic_conversion = copy.deepcopy(mne_epochs)

# Do the manuel conversion
with io.capture_output():
    manual_conversion.filter(l_freq= 8,
                      h_freq= 30,
                      picks= "all",
                      phase= "minimum",
                      fir_window= "blackman",
                      fir_design= "firwin",
                      pad= 'median', 
                      n_jobs= -1,
                      verbose= False)
    # Get a half second window
    manual_result = manual_conversion.get_data(tmin=0.1, tmax=0.6)
print(f"Manual conversion gave the following output shape: {np.shape(manual_result)}")

# Do the automatic conversion
automatic_conversion = automatic_conversion.get_data() # Works on data rather then epoch object
custom_filter = EpochsToFilteredData(start_offset = -1,
                                     data_tmin = 0.1,
                                     data_tmax = 0.6,
                                     sfreq= 200,
                                     filter_lower_bound= 8,
                                     filter_upper_bound= 30)
custom_filter.fit(automatic_conversion)
automatic_result = custom_filter.transform(automatic_conversion)
print(f"Automatic conversion gave the following output shape: {np.shape(automatic_result)}")

# Compare results
manual_result = np.array(manual_result)
automatic_result = np.array(automatic_result)
if ((automatic_result.shape == manual_result.shape) and (automatic_result == manual_result).all() ):
    match = True
else:
    match = False
print(f"results are equal: {match}")

# Remove unused variables
del automatic_conversion
del automatic_result
del manual_conversion
del manual_result
del custom_filter
del match
del mne_epochs

<hr>

### Same subject, same session: LDA classifier

As discussed in the master's thesis, training and testing a classification system can happen using multiple strategies.
A classifier may be trained on a singular subject, using a singular session and testing on that same session.
This is an over-optimistic testing scenario and has a great risk of overfitting with poor generalisation to new sessions or new subjects but can be an okay baseline test to see if *at least something* can be learned.

This experiment works as follows:
   - We use participants with at least three recordings
      - Participants: B, C, E
      - NOTE: participant F has three files provided but one of those files has only three MI classes rather than three, hence it is not considered here
   - We use the first two recorded session of each of these participants for training and the last for testing.
      - Thus, the CV scores are on the test split for the training data whilst the independent test set is from the unseen part of the session not used during training. This avoids data leakage.
   - We get epochs of 3 seconds, which includes one second before and after the visual queue
      - We use only a half a second window taking into account the online system will use sliding windows.
      - This window starts at 0.1 seconds after then visual queue and ends at 0.6 seconds after the visual queue
   - We split the data in a train/test dataset with 20% test data balanced over all MI classes
   - We use grid search on the created window of each baseline corrected epoch from the train split to find the best parameters for the pipeline
      - The pipeline that is hyperparameter tuned is as follows
         - Filter -> CSP -> LDA
      - The following hyperparameters are tested
         - For the filter:
            - Lower threshold: 1 | 2 | 4 | 6 | 8 | 10 
            - Upper threshold: 26 | 28 | 30 | 32 | 34
         - For CSP:
            - Number of components: 4 | 6 | 10
         - For LDA:
            - The optimizer: svd | lsqr
            - When using SVD optimizer, the tol is set to 0.0001, which is the default and was the best for all previous experiments
            - Shrinkage is fixed to None
            - the priors are initialized to a balanced 1/3
   - We use the test split for final validation on the best-found parameters

#### Experiment

In [ ]:
####################################################
# GRID SEARCHING BEST PIPELINE FOR EACH SUBJECT
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = (None, 0) # Baseline correction using data before the visual queue
do_experiment = True # Long experiment disabled per default

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                with io.capture_output():
                        # Get MNE raw object for latest recording of that subject
                        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id= subject_id)
                        # Get epochs for that MNE raw
                        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                             start_offset= start_offset,
                                                                             end_offset= end_offset,
                                                                             baseline= baseline)
                        
                        # Only keep epochs from the MI tasks
                        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                        
                        # Load epochs into memory
                        mne_epochs.load_data()
                        
                        # Get the labels
                        labels = mne_epochs.events[:, -1]
                        
                        # Create a test and train split
                        X_train, X_test, y_train, y_test = train_test_split(mne_epochs,
                                                                            labels,
                                                                            test_size = 0.2,
                                                                            shuffle= True,
                                                                            stratify= labels,                                                    
                                                                            random_state= 1998)
                        
                        # make split back to MNE epoch object and then retreive only the data
                        X_train = mne.concatenate_epochs(X_train).get_data()
                        X_test = mne.concatenate_epochs(X_test).get_data()
                        
                
                # Configure the pipeline components by specifying the default parameters
                custom_filter = EpochsToFilteredData(start_offset = -1,
                                                     data_tmin = 0.1,
                                                     data_tmax = 0.6,
                                                     sfreq= 200)
                
                csp = CSP(norm_trace=False,
                          component_order="mutual_info",
                          cov_est= "epoch")
                
                lda = LinearDiscriminantAnalysis(shrinkage= None,
                                                 priors=[1/3, 1/3, 1/3])
                
                # Configure the pipeline
                pipeline = Pipeline([('filter', custom_filter), ('CSP', csp), ('LDA', lda)])
                
                # Configure cross validation to use
                cv = StratifiedKFold(n_splits=4,
                                     shuffle= True,
                                     random_state= 2022)
                
                # Configure the hyperparameters to test
                # NOTE: these are somewhat limited due to limitedd computational resources
                param_grid = [{"CSP__n_components": [4, 6, 10],
                               "LDA__solver": ["svd"],
                               "LDA__tol": [0.0001],
                               "filter__filter_lower_bound": [1, 2, 4, 6, 8, 10],
                               "filter__filter_upper_bound": [26, 28, 30, 32, 34],
                               },
                              {"CSP__n_components": [4, 6, 10],
                               "LDA__solver": ["lsqr"],
                               "filter__filter_lower_bound": [1, 2, 4, 6, 8, 10],
                               "filter__filter_upper_bound": [26, 28, 30, 32, 34],
                               }]
                               
                # Configure the grid search
                grid_search = GridSearchCV(estimator= pipeline,
                                           param_grid= param_grid,
                                           scoring= "accuracy",
                                           refit= False, # We will do this manually
                                           cv= cv,
                                           verbose= 10,
                                           n_jobs= -1,
                                           return_train_score= True,
                                           error_score="raise")

                # Do the grid search on the training data
                grid_search.fit(X= X_train, y= y_train)
    
                # Store the results of the grid search
                with open(f"saved_variables/3/samesubject_samesession/subject{subject_id}/gridsearch_autofreqcsplda.pickle", 'wb') as file:
                        pickle.dump(grid_search, file)
                
                # Store the train and test data so the best model can be retrained later
                with open(f"saved_variables/3/samesubject_samesession/subject{subject_id}/testdata-x_autofreqcsplda.pickle", 'wb') as file:
                        pickle.dump(X_test, file)
                with open(f"saved_variables/3/samesubject_samesession/subject{subject_id}/testdata-y_autofreqcsplda.pickle", 'wb') as file:
                        pickle.dump(y_test, file)
                with open(f"saved_variables/3/samesubject_samesession/subject{subject_id}/traindata-x_autofreqcsplda.pickle", 'wb') as file:
                        pickle.dump(X_train, file)
                with open(f"saved_variables/3/samesubject_samesession/subject{subject_id}/traindata-y_autofreqcsplda.pickle", 'wb') as file:
                        pickle.dump(y_train, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del csp
                del lda
                del custom_filter
                del pipeline
                del labels
                del cv
                del file
                del X_train
                del X_test
                del y_train
                del y_test 
                del grid_search
                del param_grid
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset

#### Results

| **Subject** | **CSP + LDA: cross validation accuracy** | **CSP + LDA: test split accuracy** | **Config**                                          |
|-------------|------------------------------------------|------------------------------------|-----------------------------------------------------|
| B           | 0.6615 +- 0.0504                         | 0.6094                             | 6 CSP components \| LDA SVD solver with 0.0001 tol  |
| C           | 0.7144 +- 0.0341                         | 0.7240                             | 10 CSP components \| LDA SVD solver with 0.0001 tol |
| E           | 0.7342 +- 0.0171                         | 0.7277                             | 10 CSP components \| LDA SVD solver with 0.0001 tol |

In [ ]:
####################################################
# GRID SEARCH RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings

# Loop over all found results
for subject_id in subject_ids_to_test:
    print("\n\n")
    print("####################################################")
    print(f"# GRID SEARCH RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print("\n\n")
    
    # Open from file
    with open(f"saved_variables/3/samesubject_samesession/subject{subject_id}/gridsearch_autofreqcsplda.pickle", 'rb') as f:
        grid_search = pickle.load(f)
        
    # Print the results
    print(f"Best estimator has accuracy of {np.round(grid_search.best_score_, 4)} +- {np.round(grid_search.cv_results_['std_test_score'][grid_search.best_index_], 4)} with the following parameters")
    print(grid_search.best_params_)
    
    
    # Get grid search results
    grid_search_results = pd.DataFrame(grid_search.cv_results_)

    # Keep relevant columns and sort on rank
    grid_search_results.drop(labels='params', axis=1, inplace= True)
    grid_search_results.sort_values(by=['rank_test_score'], inplace=True)

    # Display grid search resulst
    print("\n\n Top 10 grid search results: ")
    display(grid_search_results.head(10))
    print("\n\n Worst 10 grid search results: ")
    display(grid_search_results.tail(10))

    # Display some statistics
    print(f"\n\nIn total there are {len(grid_search_results)} different configurations tested.")
    max_score = grid_search_results['mean_test_score'].max()
    print(f"The best mean test score is {round(max_score, 4)}")
    shared_first_place_count = len(grid_search_results[grid_search_results['mean_test_score'].between(max_score, max_score)])
    print(f"There are {shared_first_place_count} configurations with this maximum score")
    close_first_place_count = len(grid_search_results[grid_search_results['mean_test_score'].between(max_score-0.02, max_score)])
    print(f"There are {close_first_place_count} configurations within 0.02 of this maximum score")

    # Display statistics for best classifiers
    print("\n\nThe describe of the configurations within 0.02 of this maximum score is as follows:")
    display(grid_search_results[grid_search_results['mean_test_score'].between(max_score-0.02, max_score)].describe(include="all"))


# Remove unsused variables
del f
del grid_search
del max_score
del shared_first_place_count
del close_first_place_count
del grid_search_results
del subject_ids_to_test
del subject_id

In [ ]:
####################################################
# TEST RESULTS FOR BEST FOUND GRID SEARCH
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
best_found_csp_components = [4, 10 , 10]
best_found_solver = ["svd", "svd", "svd"]
best_found_tol = [0.0001, 0.0001, 0.0001]
best_found_filter_lower_bound = [2, 2, 2]
best_found_filter_upper_bound = [32, 32, 32]

# Loop over all found results
for i in range(len(subject_ids_to_test)):
    print("\n\n")
    print("####################################################")
    print(f"# TEST RESULTS FOR SUBJECT {subject_ids_to_test[i]}")
    print("####################################################")
    print("\n\n")
    
    # Open train and test data from file
    with open(f"saved_variables/3/samesubject_samesession/subject{subject_ids_to_test[i]}/testdata-x_autofreqcsplda.pickle", 'rb') as f:
        X_test = pickle.load(f)
    with open(f"saved_variables/3/samesubject_samesession/subject{subject_ids_to_test[i]}/testdata-y_autofreqcsplda.pickle", 'rb') as f:
        y_test = pickle.load(f)
    with open(f"saved_variables/3/samesubject_samesession/subject{subject_ids_to_test[i]}/traindata-x_autofreqcsplda.pickle", 'rb') as f:
        X_train = pickle.load(f)
    with open(f"saved_variables/3/samesubject_samesession/subject{subject_ids_to_test[i]}/traindata-y_autofreqcsplda.pickle", 'rb') as f:
        y_train = pickle.load(f)
        
    # Make the classifier
    custom_filter = EpochsToFilteredData(start_offset = -1,
                                         data_tmin = 0.1,
                                         data_tmax = 0.6,
                                         sfreq= 200,
                                         filter_lower_bound= best_found_filter_lower_bound[i],
                                         filter_upper_bound= best_found_filter_upper_bound[i])
    
    csp = CSP(norm_trace=False,
              component_order="mutual_info",
              cov_est= "epoch",
              n_components= best_found_csp_components[i])
    
    lda = LinearDiscriminantAnalysis(shrinkage= None,
                                     priors=[1/3, 1/3, 1/3],
                                     solver= best_found_solver[i],
                                     tol= best_found_tol[i])
    
    # Configure the pipeline
    pipeline = Pipeline([('filter', custom_filter), ('CSP', csp), ('LDA', lda)])
    
    # Fit the pipeline
    with io.capture_output():
        pipeline.fit(X_train, y_train)
    
    # Get accuracy for single fit
    y_pred = pipeline.predict(X_test)
    accuracy =  accuracy_score(y_test, y_pred)
    
    # Print accuracy results and CM
    print(f"Test accuracy for subject {subject_ids_to_test[i]}: {np.round(accuracy, 4)}")
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
        
    # plot CSP patterns estimated on train data for visualization
    pipeline['CSP'].plot_patterns(CLA_dataset.get_last_raw_mne_data_for_subject(subject_id= subject_ids_to_test[i]).info, ch_type='eeg', units='Patterns (AU)', size=1.5)    
    plt.show()


# Remove unsused variables
del subject_ids_to_test
del best_found_csp_components
del best_found_solver
del best_found_tol
del best_found_filter_lower_bound
del best_found_filter_upper_bound
del i
del f
del X_test
del y_test
del X_train
del y_train
del csp
del lda
del pipeline
del y_pred
del accuracy
